In [ ]:
import pandas as pd
import numpy as np
import glob

# 메모리 사용량 확인 및 정리

In [ ]:
def check_memory_usage():
    memory_usage = pd.DataFrame(columns=['Name', 'Memory Usage (MB)'])
    for var_name in dir():
        if var_name.startswith('_') or var_name in ['memory_usage', 'check_memory_usage', 'gc', 'pd', 'np']:
            continue
        var = globals()[var_name]
        if isinstance(var, pd.DataFrame):
            memory_mb = var.memory_usage(deep=True).sum() / 1024 / 1024
            temp_df = pd.DataFrame([[var_name, f"{memory_mb:.2f}"]], columns=['Name', 'Memory Usage (MB)'])
            memory_usage = pd.concat([memory_usage, temp_df], ignore_index=True)

    memory_usage = memory_usage.sort_values(by='Memory Usage (MB)', ascending=False).reset_index(drop=True)
    print(memory_usage.head(10))
    total_memory = float(memory_usage['Memory Usage (MB)'].str.replace(',', '').astype(float).sum())
    print(f"Total Memory Usage: {total_memory:.2f} MB")

# 파일 불러오기

In [ ]:
def load_data(data_type):
    dfs = {}
    base_path = f"/kaggle/input/credit-card-segment/open/{data_type}"
    categories = {
        "customer": "1",
        "credit": "2",
        "sales": "3",
        "billing": "4",
        "balance": "5",
        "channel": "6",
        "marketing": "7",
        "performance": "8"
    }
    for name, prefix in categories.items():
        path =  f"{base_path}/{prefix}.*/*.parquet"
        files = sorted(glob.glob(path))
        if not files:
            print(f"No parquet files found in {path}")
            continue
        dfs[name] = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)
        print(f"{name} Loaded {dfs[name].shape} parquet files")
    return dfs

In [ ]:
train_df = load_data("train")

In [ ]:
test_df = load_data("test")

In [ ]:
customer_train_df = train_df["customer"]
credit_train_df   = train_df["credit"]
sales_train_df    = train_df["sales"]
billing_train_df  = train_df["billing"]
balance_train_df  = train_df["balance"]
channel_train_df  = train_df["channel"]
marketing_train_df= train_df["marketing"]
performance_train_df = train_df["performance"]

In [ ]:
customer_test_df = test_df["customer"]
credit_test_df   = test_df["credit"]
sales_test_df    = test_df["sales"]
billing_test_df  = test_df["billing"]
balance_test_df  = test_df["balance"]
channel_test_df  = test_df["channel"]
marketing_test_df= test_df["marketing"]
performance_test_df = test_df["performance"]

In [ ]:
def check_memory_usage(df):
    memory_usage = df.memory_usage(deep=True).sum() / 1024**2  # MB 단위로 변환
    print(f"메모리 사용량: {memory_usage:.2f} MB")
    return memory_usage

In [ ]:
print("메모리 사용량 확인:")
check_memory_usage(customer_train_df)

In [ ]:
import gc

In [ ]:
import re

# 전처리 함수들
1. '개, 대, 회, 이상, 회이상, 개이상, 대이상, 대 이상, 회 이상'로 끝나는 문자열에서 숫자만 추출
2. '', 'none', 'null', 'nan', 'NaN' 이 문자열로 들어왔을 때 처리, 10101, -999999, -99, 999, 99999999 값 0 처리
3. unique 값이 1개인 컬럼 drop
4. IQR 방식으로 이상치 탐색 후 NaN 으로 대치 (날짜 형태의 컬럼은 0으로 처리)
5. 날짜형 데이터의 추가적인 결측치가 있을 경우 0처리
6. 각 컬럼별 결측치의 비율에 따라 평균값, 중앙값, 최빈값, drop등으로 처리

## 문자열 -> 수치형으로 파싱

'개, 대, 회, 이상, 회이상, 개이상, 대이상'로 끝나는 문자열에서 숫자만 추출


In [ ]:
def clean_units_from_dataframe(df):
    # 데이터프레임 복사본 생성
    df_cleaned = df.copy()
    object_columns = df_cleaned.select_dtypes(include=['object']).columns
    # 각 object 컬럼 처리
    for col in object_columns:
        # 문자열인 경우에만 처리
        mask = df_cleaned[col].apply(lambda x: isinstance(x, str))

        if mask.any():  # 문자열이 하나 이상 있는 경우만 처리
            unit_pattern = r'\d+\s*(개|대|회|이상|회이상|개이상|대이상|회 이상|일 이상)$'
            # re.search를 직접 적용하여 경고 피하기
            unit_mask = df_cleaned.loc[mask, col].apply(lambda x: bool(re.search(unit_pattern, x)))

            # 단위가 있는 값만 처리
            if unit_mask.any():
                # 숫자 추출하기 위한 패턴
                number_pattern = r'(\d+)\s*(개|대|회|이상|회이상|개이상|대이상|회 이상|일 이상)$'
                # 숫자 부분만 추출
                extracted_numbers = df_cleaned.loc[mask, col].str.extract(number_pattern)[0]
                # 추출된 숫자를 새로운 값으로 사용 (숫자로 변환)
                df_cleaned.loc[mask & unit_mask, col] = pd.to_numeric(
                    extracted_numbers.loc[unit_mask], errors='coerce')
    return df_cleaned

In [ ]:
customer_train_df = clean_units_from_dataframe(customer_train_df)
credit_train_df   = clean_units_from_dataframe(credit_train_df)
sales_train_df    = clean_units_from_dataframe(sales_train_df)
billing_train_df  = clean_units_from_dataframe(billing_train_df)
balance_train_df  = clean_units_from_dataframe(balance_train_df)
channel_train_df  = clean_units_from_dataframe(channel_train_df)
marketing_train_df= clean_units_from_dataframe(marketing_train_df)
performance_train_df = clean_units_from_dataframe(performance_train_df)

In [ ]:
customer_test_df     = clean_units_from_dataframe(customer_test_df)
credit_test_df       = clean_units_from_dataframe(credit_test_df)
sales_test_df        = clean_units_from_dataframe(sales_test_df)
billing_test_df      = clean_units_from_dataframe(billing_test_df)
balance_test_df      = clean_units_from_dataframe(balance_test_df)
channel_test_df      = clean_units_from_dataframe(channel_test_df)
marketing_test_df    = clean_units_from_dataframe(marketing_test_df)
performance_test_df  = clean_units_from_dataframe(performance_test_df)

In [ ]:
marketing_test_df.head()

## 결측치 수기 처리

'', 'none', 'null', 'nan', 'NaN' 이 문자열로 들어왔을 때 처리, 10101, -999999, -99, 999, 99999999 값 0 처리

In [ ]:
def is_special_null(df):
    df_copy = df.copy()

    # 문자열 컬럼 처리 (벡터화된 방식)
    for col in df_copy.select_dtypes(include=['object']).columns:
        # 문자열 컬럼에서 빈 문자열, 'none', 'null', 'nan' 찾기
        mask = df_copy[col].astype(str).str.lower().isin(['', 'none', 'null', 'nan', 'NaN'])
        df_copy.loc[mask, col] = np.nan  # NaN으로 변경

    # 숫자 컬럼 처리
    for col in df_copy.select_dtypes(include=['int64', 'float64']).columns:
        # 특정 숫자값 찾기
        mask = df_copy[col].isin([10101, -999999, -99, 999, 99999999])
        df_copy.loc[mask, col] = np.nan  # NaN으로 변경
    return df_copy

In [ ]:
customer_train_df = is_special_null(customer_train_df)
credit_train_df   = is_special_null(credit_train_df)
sales_train_df    = is_special_null(sales_train_df)
billing_train_df  = is_special_null(billing_train_df)
balance_train_df  = is_special_null(balance_train_df)
channel_train_df  = is_special_null(channel_train_df)
marketing_train_df= is_special_null(marketing_train_df)
performance_train_df = is_special_null(performance_train_df)

In [ ]:
customer_test_df     = is_special_null(customer_test_df)
credit_test_df       = is_special_null(credit_test_df)
sales_test_df        = is_special_null(sales_test_df)
billing_test_df      = is_special_null(billing_test_df)
balance_test_df      = is_special_null(balance_test_df)
channel_test_df      = is_special_null(channel_test_df)
marketing_test_df    = is_special_null(marketing_test_df)
performance_test_df  = is_special_null(performance_test_df)

In [ ]:
marketing_test_df.head()

## 컬럼별 데이터 형태가 1개밖에 없는 경우 drop

unique 값이 1개인 컬럼 drop


In [ ]:
def remove_single_value_columns(df):
    single_value_cols = []
    for col in df.columns:
        if df[col].nunique() == 1:
            single_value_cols.append(col)

    if single_value_cols:
        print(f"제거할 컬럼 (유니크 값 1개): {single_value_cols}")
        df = df.drop(columns=single_value_cols)
    else:
        print("유니크 값이 1개인 컬럼이 없습니다.")

    return df

In [ ]:
customer_train_df = remove_single_value_columns(customer_train_df)
credit_train_df   = remove_single_value_columns(credit_train_df)
sales_train_df    = remove_single_value_columns(sales_train_df)
billing_train_df  = remove_single_value_columns(billing_train_df)
balance_train_df  = remove_single_value_columns(balance_train_df)
channel_train_df  = remove_single_value_columns(channel_train_df)
marketing_train_df= remove_single_value_columns(marketing_train_df)
performance_train_df = remove_single_value_columns(performance_train_df)

In [ ]:
customer_test_df     = remove_single_value_columns(customer_test_df)
credit_test_df       = remove_single_value_columns(credit_test_df)
sales_test_df        = remove_single_value_columns(sales_test_df)
billing_test_df      = remove_single_value_columns(billing_test_df)
balance_test_df      = remove_single_value_columns(balance_test_df)
channel_test_df      = remove_single_value_columns(channel_test_df)
marketing_test_df    = remove_single_value_columns(marketing_test_df)
performance_test_df  = remove_single_value_columns(performance_test_df)

In [ ]:
customer_date_columns = ["입회일자_신용", "최종유효년월_신용_이용가능", "최종유효년월_신용_이용", "최종카드발급일자"]
sales_date_columns = ["최종이용일자_기본", "최종이용일자_신판", "최종이용일자_CA", "최종이용일자_카드론", "최종이용일자_체크", "최종이용일자_일시불", "최종이용일자_할부", "최종카드론_대출일자"]

# 입회일자_신용, 최종유효년월_신용_이용가능, 최종유효년월_신용_이용, 최종카드발급일자
#
# 최종이용일자_기본, 최종이용일자_신판, 최종이용일자_CA, 최종이용일자_카드론, 최종이용일자_체크, 최종이용일자_일시불, 최종이용일자_할부, 최종카드론_대출일자
#
#
#
#

## 이상치 처리

IQR 방식으로 이상치 탐색 후 NaN 으로 대치 (날짜 형태의 컬럼은 0으로 처리)

날짜형 데이터의 추가적인 결측치가 있을 경우 0처리

In [ ]:
def handle_outliers_with_iqr(df, date_columns=None):
    df_copy = df.copy()

    if date_columns is None:
        date_columns = []

    # 모든 수치형 컬럼 선택
    numeric_cols = df_copy.select_dtypes(include=['int64', 'float64']).columns

    # 각 수치형 컬럼 처리
    for col in numeric_cols:
        # 결측치가 아닌 값으로만 IQR 계산
        valid_data = df_copy[col].dropna()

        if len(valid_data) > 0:  # 유효한 데이터가 있는 경우만 처리
            Q1 = valid_data.quantile(0.25)
            Q3 = valid_data.quantile(0.75)
            IQR = Q3 - Q1

            # 이상치 경계 계산
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # 이상치 찾기
            mask_lower = df_copy[col] < lower_bound
            mask_upper = df_copy[col] > upper_bound

            # 이상치 개수 출력
            outliers_count = mask_lower.sum() + mask_upper.sum()
            if outliers_count > 0:
                print(f"컬럼 '{col}': {outliers_count}개의 이상치 발견 (하한값: {lower_bound}, 상한값: {upper_bound})")

            # 이상치 처리 - date_columns에 포함된 컬럼이면 0으로, 아니면 NaN으로
            if col in date_columns:
                df_copy.loc[mask_lower | mask_upper, col] = 0
                if outliers_count > 0:
                    print(f"  -> 날짜 컬럼으로 간주하여 이상치를 0으로 대체")
            else:
                df_copy.loc[mask_lower | mask_upper, col] = np.nan
                if outliers_count > 0:
                    print(f"  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체")

    return df_copy

In [ ]:
customer_train_df   = handle_outliers_with_iqr(customer_train_df, customer_date_columns)
credit_train_df      = handle_outliers_with_iqr(credit_train_df)
sales_train_df      = handle_outliers_with_iqr(sales_train_df, sales_date_columns)
billing_train_df    = handle_outliers_with_iqr(billing_train_df)
balance_train_df    = handle_outliers_with_iqr(balance_train_df)
channel_train_df    = handle_outliers_with_iqr(channel_train_df)
marketing_train_df  = handle_outliers_with_iqr(marketing_train_df)
performance_train_df = handle_outliers_with_iqr(performance_train_df)

In [ ]:
customer_test_df     = handle_outliers_with_iqr(customer_test_df, customer_date_columns)
credit_test_df       = handle_outliers_with_iqr(credit_test_df)
sales_test_df        = handle_outliers_with_iqr(sales_test_df, sales_date_columns)
billing_test_df      = handle_outliers_with_iqr(billing_test_df)
balance_test_df      = handle_outliers_with_iqr(balance_test_df)
channel_test_df      = handle_outliers_with_iqr(channel_test_df)
marketing_test_df    = handle_outliers_with_iqr(marketing_test_df)
performance_test_df  = handle_outliers_with_iqr(performance_test_df)

In [ ]:
def handle_date_columns(df, date_cols):
    for col in date_cols:
        if col in df.columns:
            missing_before = df[col].isnull().sum()
            df[col] = df[col].fillna(0)
            missing_after = df[col].isnull().sum()
            print(f"컬럼 '{col}'의 결측치: {missing_before} -> {missing_after}")
    return df

In [ ]:
customer_train_df = handle_date_columns(customer_train_df, customer_date_columns)

sales_train_df = handle_date_columns(sales_train_df, sales_date_columns)

In [ ]:
customer_test_df = handle_date_columns(customer_test_df, customer_date_columns)

sales_test_df = handle_date_columns(sales_test_df, sales_date_columns)

## 결측치 처리

각 컬럼별 결측치의 비율에 따라 평균값, 중앙값, 최빈값, drop등으로 처리

In [ ]:
def handle_missing_values(df):
    df_copy = df.copy()

    # 결측치가 있는 컬럼 찾기
    columns_with_missing = [col for col in df_copy.columns if df_copy[col].isnull().sum() > 0]

    for col in columns_with_missing:
        missing_count = df_copy[col].isnull().sum()
        missing_ratio = missing_count / len(df_copy)
        print(f"컬럼 '{col}'의 결측치: {missing_count}개 ({missing_ratio:.4f})")

        # 결측 비율에 따른 처리
        if missing_ratio > 0.75:
            # 75% 초과 결측: 컬럼 제거 고려
            print(f"  -> 결측치 비율이 75%를 초과합니다. 해당 컬럼 제거")
            df_copy = df_copy.drop(columns=[col])

        elif missing_ratio > 0.5:
            # 50~75% 결측: 컬럼 유지하되 결측치는 분포의 중심값으로 대체
            if df_copy[col].dtype in ['int64', 'float64']:
                # 수치형: 중앙값(median)으로 대체
                median_val = df_copy[col].median()
                df_copy[col] = df_copy[col].fillna(median_val)
                print(f"  -> 결측 비율이 높은 수치형 컬럼이므로 중앙값 {median_val}으로 대체")
            else:
                # 범주형: 최빈값(mode)으로 대체
                mode_val = df_copy[col].mode()[0] if not df_copy[col].mode().empty else "unknown"
                df_copy[col] = df_copy[col].fillna(mode_val)
                print(f"  -> 결측 비율이 높은 범주형 컬럼이므로 최빈값 '{mode_val}'으로 대체")

        elif missing_ratio > 0.3:
            # 30~50% 결측: 수치형은 평균/분포 기반, 범주형은 최빈값 대체
            if df_copy[col].dtype in ['int64', 'float64']:
                # 왜도(skewness) 확인하여 처리
                skew = df_copy[col].skew()
                if abs(skew) > 1:  # 비대칭 분포인 경우
                    median_val = df_copy[col].median()
                    df_copy[col] = df_copy[col].fillna(median_val)
                    print(f"  -> 분포가 비대칭(skew={skew:.2f})이므로 중앙값 {median_val}으로 대체")
                else:  # 대칭 분포인 경우
                    mean_val = df_copy[col].mean()
                    df_copy[col] = df_copy[col].fillna(mean_val)
                    print(f"  -> 분포가 대칭(skew={skew:.2f})이므로 평균값 {mean_val}으로 대체")
            else:
                # 범주형: 최빈값으로 대체
                mode_val = df_copy[col].mode()[0] if not df_copy[col].mode().empty else "unknown"
                df_copy[col] = df_copy[col].fillna(mode_val)
                print(f"  -> 범주형 컬럼이므로 최빈값 '{mode_val}'으로 대체")

        else:
            # 30% 이하 결측: 일반적인 대체 방법 적용
            if df_copy[col].dtype in ['int64', 'float64']:
                # 왜도 확인
                skew = df_copy[col].skew()
                if abs(skew) > 1:
                    median_val = df_copy[col].median()
                    df_copy[col] = df_copy[col].fillna(median_val)
                    print(f"  -> 분포가 비대칭(skew={skew:.2f})이므로 중앙값 {median_val}으로 대체")
                else:
                    mean_val = df_copy[col].mean()
                    df_copy[col] = df_copy[col].fillna(mean_val)
                    print(f"  -> 분포가 대칭(skew={skew:.2f})이므로 평균값 {mean_val}으로 대체")
            else:
                mode_val = df_copy[col].mode()[0] if not df_copy[col].mode().empty else "unknown"
                df_copy[col] = df_copy[col].fillna(mode_val)
                print(f"  -> 범주형 컬럼이므로 최빈값 '{mode_val}'으로 대체")

    return df_copy

In [ ]:
customer_train_df   = handle_missing_values(customer_train_df)
credit_train_df      = handle_missing_values(credit_train_df)
sales_train_df      = handle_missing_values(sales_train_df)
billing_train_df    = handle_missing_values(billing_train_df)
balance_train_df    = handle_missing_values(balance_train_df)
channel_train_df    = handle_missing_values(channel_train_df)
marketing_train_df  = handle_missing_values(marketing_train_df)
performance_train_df = handle_missing_values(performance_train_df)

In [ ]:
customer_test_df     = handle_missing_values(customer_test_df)
credit_test_df       = handle_missing_values(credit_test_df)
sales_test_df        = handle_missing_values(sales_test_df)
billing_test_df      = handle_missing_values(billing_test_df)
balance_test_df      = handle_missing_values(balance_test_df)
channel_test_df      = handle_missing_values(channel_test_df)
marketing_test_df    = handle_missing_values(marketing_test_df)
performance_test_df  = handle_missing_values(performance_test_df)

# 라벨 인코딩 처리

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
def label_encode_dataframe(df, exclude_cols=None):
    # 원본 데이터프레임을 수정하지 않기 위해 복사
    df_encoded = df.copy()

    # 제외할 컬럼 목록 설정
    if exclude_cols is None:
        exclude_cols = []

    # object 타입 컬럼 찾기
    object_columns = df_encoded.select_dtypes(include=['object']).columns

    # 인코딩할 컬럼 (제외 컬럼 제외)
    encode_columns = [col for col in object_columns if col not in exclude_cols]

    if len(encode_columns) == 0:
        print("인코딩할 object 타입 컬럼이 없습니다.")
        return df_encoded

    print(f"총 {len(encode_columns)}개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.")

    # 각 컬럼에 라벨 인코딩 적용
    for col in encode_columns:
        print(f"컬럼 '{col}' 인코딩 중...")

        # 해당 컬럼의 유니크 값 수 확인
        unique_count = df_encoded[col].nunique()
        print(f"  -> 유니크 값 수: {unique_count}")

        # 누락된 값이 있는지 확인
        null_count = df_encoded[col].isnull().sum()
        if null_count > 0:
            print(f"  -> 주의: {null_count}개의 결측치가 있습니다.")

        # 라벨 인코더 생성 및 적용
        le = LabelEncoder()

        # 결측치가 있는 경우 임시로 처리 (인코딩 후 다시 NaN으로 변경)
        temp_col = df_encoded[col].copy()
        mask_null = temp_col.isnull()

        if mask_null.any():
            # 결측치는 임시로 특수 문자열로 대체
            temp_col = temp_col.fillna('NULL_VALUE_FOR_ENCODING')

        # 인코딩 적용
        temp_col = le.fit_transform(temp_col.astype(str))

        # 인코딩된 값 데이터프레임에 대입
        df_encoded[col] = temp_col

        # 결측치가 있었다면 다시 NaN으로 변경
        if mask_null.any():
            df_encoded.loc[mask_null, col] = np.nan

        print(f"  -> 인코딩 완료: {len(le.classes_)}개의 클래스로 변환")

    print("모든 컬럼 인코딩 완료!")
    return df_encoded  # 인코딩된 데이터프레임만 반환

In [ ]:
exclude_columns = ['ID', 'Segment']

customer_train_df   = label_encode_dataframe(customer_train_df, exclude_columns)
credit_train_df = label_encode_dataframe(credit_train_df, exclude_columns)
sales_train_df = label_encode_dataframe(sales_train_df, exclude_columns)
billing_train_df = label_encode_dataframe(billing_train_df, exclude_columns)
channel_train_df = label_encode_dataframe(channel_train_df, exclude_columns)
marketing_train_df = label_encode_dataframe(marketing_train_df, exclude_columns)
performance_train_df = label_encode_dataframe(performance_train_df, exclude_columns)

In [ ]:
exclude_columns = ['ID']

customer_test_df   = label_encode_dataframe(customer_test_df, exclude_columns)
credit_test_df     = label_encode_dataframe(credit_test_df, exclude_columns)
sales_test_df      = label_encode_dataframe(sales_test_df, exclude_columns)
billing_test_df    = label_encode_dataframe(billing_test_df, exclude_columns)
channel_test_df    = label_encode_dataframe(channel_test_df, exclude_columns)
marketing_test_df  = label_encode_dataframe(marketing_test_df, exclude_columns)
performance_test_df= label_encode_dataframe(performance_test_df, exclude_columns)

In [ ]:
customer_test_df

In [ ]:
# 기본 데이터프레임 설정
merged_train_df = customer_train_df.copy()

# 나머지 데이터프레임들을 병합
for df_name in ['credit_train_df', 'sales_train_df', 'billing_train_df',
                'channel_train_df', 'marketing_train_df', 'performance_train_df']:
    print(f"{df_name} 병합 중...")
    # '기준년월'과 'ID' 모두를 기준으로 병합
    merged_train_df = merged_train_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"현재 병합된 데이터프레임 크기: {merged_train_df.shape}")
    del globals()[df_name]
    gc.collect()

In [ ]:
merged_test_df = customer_test_df.copy()

# 나머지 데이터프레임들을 병합
for df_name in ['credit_test_df', 'sales_test_df', 'billing_test_df',
                'channel_test_df', 'marketing_test_df', 'performance_test_df']:
    print(f"{df_name} 병합 중...")
    # '기준년월'과 'ID' 모두를 기준으로 병합
    merged_test_df = merged_test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"현재 병합된 데이터프레임 크기: {merged_test_df.shape}")
    del globals()[df_name]
    gc.collect()

In [ ]:
import xgboost as xgb

In [ ]:
feature_cols = [col for col in merged_train_df.columns if col not in ["ID", "Segment"]]

X = merged_train_df[feature_cols].copy()
y = merged_train_df["Segment"].copy()

le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [ ]:
X_test = merged_test_df[feature_cols].copy()

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
for col in categorical_features:
    # 결측치 채우기 (있는 경우)
    if X[col].isnull().any():
        X[col] = X[col].fillna('-999')
    if X_test[col].isnull().any():
        X_test[col] = X_test[col].fillna('-999')
    
    # 테스트 데이터에 없는 값이 있으면 처리
    all_values = set(X[col].unique()) | set(X_test[col].unique())
    label_map = {val: idx for idx, val in enumerate(all_values)}
    
    X[col] = X[col].map(label_map)
    X_test[col] = X_test[col].map(label_map)

In [ ]:
gc.collect()

# 학습

In [ ]:
try:
    model = xgb.XGBClassifier(
        tree_method='gpu_hist',  # GPU 모드 설정
        gpu_id=0,
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    print("GPU 사용 가능: gpu_hist 모드 적용")
    model.fit(X, y_encoded)
except Exception:
    model = xgb.XGBClassifier(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    print("GPU 사용 불가: CPU 모드 적용")
    model.fit(X, y_encoded)

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
y_test_pred_labels = le_target.inverse_transform(y_test_pred)

In [ ]:
# 예측 결과를 테스트 데이터에 추가
test_data = merged_test_df[["ID"]].copy()
test_data["pred_label"] = y_test_pred_labels

In [ ]:
# ID별로 가장 많이 예측된 세그먼트 선택
submission = test_data.groupby("ID")["pred_label"] \
             .agg(lambda x: x.value_counts().idxmax()) \
             .reset_index()
submission.columns = ["ID", "Segment"]

In [ ]:
# 결과 저장
submission.to_csv('./submission.csv', index=False)
print("예측 완료! 제출 파일이 생성되었습니다.")

In [ ]:
# 성능 지표 확인 (상위 20개 중요 특성)
if hasattr(model, 'feature_importances_'):
    feature_importance = pd.Series(model.feature_importances_, index=X.columns)
    top_features = feature_importance.nlargest(20)
    print("\n상위 20개 중요 특성:")
    print(top_features)